In [109]:
import difflib
import numpy as np
from sklearn.neighbors import NearestNeighbors
import Levenshtein

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False
        self.description = None

class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word, description):
        current = self.root
        for char in word:
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        current.is_word = True
        current.description = description
    
    def search(self, word):
        current = self.root
        for char in word:
            if char not in current.children:
                return None
            current = current.children[char]
        if current.is_word:
            return current.description
        return None
    
    def fuzzy_search(self, word, cutoff=0.6):
        results = difflib.get_close_matches(word, self.words(), n=10, cutoff=cutoff)
        return {result: (self.search(result), difflib.SequenceMatcher(None, word, result).ratio()) for result in results}

    def fuzzy_search_knn(self, word, k=5, cutoff=0.6):
        words = np.array(self.words())
        words_len = np.array([len(w) for w in words])
        word_len = len(word)
        distances = np.abs(words_len - word_len)
        knn = NearestNeighbors(n_neighbors=k, metric='manhattan')
        knn.fit(distances.reshape(-1, 1))
        _, indices = knn.kneighbors(np.array([word_len]).reshape(-1, 1))
        results = [words[index] for index in indices[0]]
        ratio = [difflib.SequenceMatcher(None, word, result).ratio() for result in results]
        return {result: (self.search(result), ratio[i]) for i, result in enumerate(results) if ratio[i] >= cutoff}
        
        
    def words(self):
        words = []
        def dfs(node, word):
            if node.is_word:
                words.append(word)
            for char in node.children:
                dfs(node.children[char], word + char)
        dfs(self.root, "")
        return words


In [110]:
trie = Trie()
trie.insert("first_name", "The first name of a person")
trie.insert("last_name", "The last name of a person")
trie.insert("street", "The street where somebody lives")
trie.insert("company", "enterprise where the employee works")
trie.insert("vehicle_type", "type of vehicle  the person owns")
trie.insert("model", "model of the car")
trie.insert("date_of_birth", "when the person was born")
trie.insert("language", "which language the person speaks")




In [111]:
print(trie.search("first_name")) # Output: The first name of a person
print(trie.fuzzy_search("srt")) # Output: {'first_name': 'The first name of a person'}
print(trie.fuzzy_search("lname"))
print(trie.fuzzy_search("strt"))
print(trie.fuzzy_search_knn("srt", k=1))

The first name of a person
{}
{'last_name': ('The last name of a person', 0.7142857142857143)}
{'street': ('The street where somebody lives', 0.8)}
{'street': ('The street where somebody lives', 0.6666666666666666)}


In [114]:
print(trie.fuzzy_search("fname"))
print(trie.fuzzy_search("FirstName"))

print(trie.fuzzy_search_knn("fname", k = 1))
print(trie.fuzzy_search_knn("FirstName", k = 2))

{'first_name': ('The first name of a person', 0.6666666666666666)}
{'first_name': ('The first name of a person', 0.7368421052631579)}
{'first_name': ('The first name of a person', 0.6666666666666666)}
{}


In [119]:
print(trie.fuzzy_search("lname"))
print(trie.fuzzy_search("lastName"))

print(trie.fuzzy_search_knn("last_name", k = 1))
print(trie.fuzzy_search_knn("lastName", k = 1))

{'last_name': ('The last name of a person', 0.7142857142857143)}
{'last_name': ('The last name of a person', 0.8235294117647058)}
{}
{}


In [134]:
import collections

def build_trie(words):
    root = collections.defaultdict(lambda: {"desc": None, "children": collections.defaultdict(dict)})
    for word, desc in words:
        node = root
        for char in word:
            node = node["children"].setdefault(char, collections.defaultdict(dict))
        node["desc"] = desc
    return root

def damerau_levenshtein_distance(a, b, d):
    la, lb = len(a), len(b)
    da = [0] * (la + 1)
    db = [0] * (lb + 1)
    for i in range(la + 1):
        da[i] = i
    for j in range(lb + 1):
        db[j] = j
    for i in range(1, la + 1):
        for j in range(1, lb + 1):
            cost = 0 if a[i - 1] == b[j - 1] else 1
            da[i] = min(da[i - 1] + 1, db[j] + 1, da[i - 1] + cost)
            if i > 1 and j > 1 and a[i - 1] == b[j - 2] and a[i - 2] == b[j - 1]:
                da[i] = min(da[i], db[j - 2] + cost)
            db[j] = da[i]
    return da[la]

def fuzzy_match(trie, pattern, max_distance):
    queue = [(trie, 0, 0, [])]
    while queue:
        node, i, distance, path = queue.pop(0)
        if distance > max_distance:
            continue
        if "desc" in node and node["desc"] is not None:
            yield "".join(path), node["desc"]
        for char, child in node["children"].items():
            j = i + 1
            if j == len(pattern):
                if distance <= max_distance:
                    yield char, node["desc"]
                continue
            if char == pattern[j]:
                queue.append((child, j, distance, path + [char]))
            else:
                d = damerau_levenshtein_distance(char, pattern[j], max_distance)
                queue.append((child, j, distance + d, path + [char]))

words = [("dog", "domestic animal"), ("doe", "female deer"), ("cat", "domestic animal"), ("cot", "a bed for camping")]
trie = build_trie(words)
for word, desc in fuzzy_match(trie, 'dot', 1):
    print(word, ":", desc)


In [139]:
result = fuzzy_match(trie, 'dot', 1)
print(result.word)

AttributeError: 'generator' object has no attribute 'word'